In [1]:
import os
import sys

sys.path.insert(0, '../..')

from absl import flags
flags.FLAGS(['notebook'])

import tensorflow as tf

from lmml.models.yolov3 import draw_labels, draw_outputs, YoloV3, yolo_outputs_to_pred, yolo_anchor_masks, yolo_anchors
from lmmlscripts.yolo.dataset import load_dataset

In [10]:
# Load val dataset
# os.environ.remove('TFDS_DATA_DIR')
import tensorflow_datasets as tfds
ds = tfds.load('waymo_open_dataset/v1.0', split='validation', data_dir='gs://waymo_open_dataset_v_1_0_0_individual_files/tensorflow_datasets')

ds = load_dataset('tfds://waymo_open_dataset/v1.0', 'validation', 4, 416)
ds_iter = iter(ds)

PermissionDeniedError: Error executing an HTTP request: HTTP response code 401 with body '{
  "error": {
    "code": 401,
    "message": "Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket.",
    "errors": [
      {
        "message": "Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket.",
        "domain": "global",
        "reason": "required",
        "locationType": "header",
        "location": "Authorization"
      }
    ]
  }
}
'
	 when reading gs://waymo_open_dataset_v_1_0_0_individual_files/training
  In call to configurable 'load_dataset' (<function load_dataset at 0x148748550>)

In [ ]:
# Load checkpoints
strategy = tf.distribute.MirroredStrategy(devices=tf.config.list_logical_devices('CPU'))
with strategy.scope():
    BACKBONE_SAVED_MODEL_PATH = '/Users/jaigupta/data/saved_models/darknet'
    YOLO_SAVED_MODEL_PATH = '/Users/jaigupta/data/saved_models/yolov3_2'
    backbone_model  = (
        tf.saved_model.load(BACKBONE_SAVED_MODEL_PATH)
        .signatures['serving_default'])

    model = (
        tf.saved_model.load(YOLO_SAVED_MODEL_PATH)
        .signatures['serving_default']
    )



In [ ]:
# run checkpoint on dataset and plot the output on image.

@tf.function
def predict(input):
    outputs = backbone_model(input_1=input)
    x_36, x_61, x = outputs['add_10'], outputs['add_18'], outputs['add_22']
    return model(f_36=x_36, f_61=x_61, f=x)

In [ ]:

class_names = ['class_' + str(i) for i in range(100)]

inp, labels = next(ds_iter)
outputs = strategy.run(predict, (inp,))
boxes, scores, classes, nums = yolo_outputs_to_pred(
    outputs["yolo_output_0"], outputs["yolo_output_1"], outputs["yolo_output_2"],
    yolo_anchors, yolo_anchor_masks, 5)
boxes

In [6]:
import cv2
from matplotlib import pyplot as plt
idx=3
img = cv2.cvtColor(inp[idx].numpy(), cv2.COLOR_RGB2BGR)
plt.figure()
plt.imshow(draw_outputs(img, (boxes[idx], scores[idx], classes[idx], nums[idx]), class_names))

res = draw_labels(inp[idx], labels[idx].numpy(), class_names)
plt.figure()
plt.imshow(res)

NameError: name 'inp' is not defined

In [7]:
boxes[idx]

<tf.Tensor: shape=(100, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
